In [10]:
import pandas as pd 
df = pd.read_csv('imdb_indonesian_movies_2.csv')

In [11]:
df.head(5)

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [12]:
# membuat dummy variable dari genre menggunakan one hot encoding
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df,category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru.head()

,judul_film,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0


In [13]:
# memisahkan label dan fitur dan menjadikannya array numpy
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama','Horor','Komedi','Laga','Romantis']].values

In [14]:
#split data training dan testing
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(sinopsis, label, test_size=0.2)
sinopsis

array(['Raden Mas Said putra sulung Tumenggung Wilarikta di bawah Kerajaan Majapahit yang berkuasa di wilayah Tuban melihat sekeluarga miskin yang menderita busung lapar. Ia merasa sangat prihatin dan hati nuraninya tergugah untuk menolong. Kemudian ia mencoba secara diam-diam mengambil makanan dari lumbung orang tuanya. Perbuatan itu tidak disetujui orang tuanya bahkan ia dihukum sekap di gudang makanan itu. Sejak kejadian itu RM Said yang tumbuh dewasa tidak betah tinggal di rumah. Ia berkelana dari daerah satu ke daerah lainnya. Dari sanalah ia tahu betapa banyak penyelewengan dan kesewenang-wenangan para lurah yang munafik. Mereka selalu mengkambing-hitamkan Tumenggung untuk menutupi kejahatannya. Atas laporan RM Said ayahnya kemudian sadar. Tetapi kemudian ia dianggap sebagai sumber fitnah. Dalam kelananya kemudian ia bertemu dengan Sunan Bonang yang banyak mencurahkan ilmunya kepada RM Said. Ia pun kemudian melakukan tapa di pinggir kali. Berkat ketabahannya menghadapi berbagai c

In [15]:
# tokenisasi dan padding sinopsis
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenisasi sinopsis training dan testing
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)

#membuat array dari data yang sudah ditokenisasi
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

#menyamakan panjang array/sequence
train_padded = pad_sequences(train_sequences)
test_padded = pad_sequences(test_sequences)

In [16]:
len(test_padded[0])

1765

In [17]:
#arsitektru model
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(62, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [19]:
#training model
history = model.fit(train_padded, y_train, validation_data=(test_padded, y_test), epochs=10)

Epoch 1/10
26/26 [==============================] - 9s 353ms/step - loss: 1.6096 - accuracy: 0.1990 - val_loss: 1.6104 - val_accuracy: 0.1940
Epoch 2/10
26/26 [==============================] - 8s 293ms/step - loss: 1.6047 - accuracy: 0.2425 - val_loss: 1.6087 - val_accuracy: 0.2189
Epoch 3/10
26/26 [==============================] - 8s 297ms/step - loss: 1.5618 - accuracy: 0.3358 - val_loss: 1.5929 - val_accuracy: 0.2786
Epoch 4/10
26/26 [==============================] - 8s 317ms/step - loss: 1.3176 - accuracy: 0.5386 - val_loss: 1.6556 - val_accuracy: 0.2736
Epoch 5/10
26/26 [==============================] - 8s 302ms/step - loss: 0.9285 - accuracy: 0.6878 - val_loss: 2.0398 - val_accuracy: 0.2687
Epoch 6/10
26/26 [==============================] - 8s 313ms/step - loss: 0.5040 - accuracy: 0.8545 - val_loss: 2.6583 - val_accuracy: 0.2587
Epoch 7/10
26/26 [==============================] - 8s 296ms/step - loss: 0.2140 - accuracy: 0.9415 - val_loss: 3.1248 - val_accuracy: 0.2736
Epoch 